This is the part 1 of the preprocessing pipeline
However, an additional cleaning (change None/"space" into Nan) should be carried beforehand

 --Written by Xingyue Huang 5th.Mar.2021

In [1]:
import pandas as pd
import numpy as np
import os
import keras

In [2]:
# Read the sample file
co_file = "Co_600K_Jul2019_6M.pkl"
df_co = pd.read_pickle(co_file)

In [3]:
print("   Column                         No of unique values")
for n,c in enumerate(df_co.columns):
    print("{:3d}. {:<35}: {:>6}".format(n,c, len(df_co[c].unique())))
    print("Number of nans: ", df_co[c].isnull().sum())
    print("type of data: ", type(df_co[c][0]))
    print()

   Column                         No of unique values
  0. AccountsAccountCategory            :     16
Number of nans:  0
type of data:  <class 'str'>

  1. AccountsAccountRefDay              :     32
Number of nans:  4
type of data:  <class 'str'>

  2. AccountsAccountRefMonth            :     13
Number of nans:  4
type of data:  <class 'str'>

  3. CompanyCategory                    :      2
Number of nans:  0
type of data:  <class 'str'>

  4. CompanyId                          : 600000
Number of nans:  0
type of data:  <class 'numpy.int64'>

  5. CompanyName                        : 600000
Number of nans:  0
type of data:  <class 'str'>

  6. CompanyNameCountNum                :     16
Number of nans:  0
type of data:  <class 'numpy.int64'>

  7. CompanyNameCountX                  :      5
Number of nans:  0
type of data:  <class 'numpy.int64'>

  8. CompanyNameLen                     :     85
Number of nans:  0
type of data:  <class 'numpy.int64'>

  9. CompanyNameWordLen         

Number of nans:  0
type of data:  <class 'numpy.int32'>

 82. eCompanyCategory                   :      2
Number of nans:  0
type of data:  <class 'numpy.int32'>

 83. hasF1014                           :      2
Number of nans:  0
type of data:  <class 'numpy.int32'>

 84. hasF1129                           :      2
Number of nans:  0
type of data:  <class 'numpy.int32'>

 85. hasF1522                           :      2
Number of nans:  0
type of data:  <class 'numpy.int32'>

 86. hasF1631                           :      2
Number of nans:  0
type of data:  <class 'numpy.int32'>

 87. hasF17                             :      2
Number of nans:  0
type of data:  <class 'numpy.int32'>

 88. hasF1865                           :      2
Number of nans:  0
type of data:  <class 'numpy.int32'>

 89. hasF1871                           :      2
Number of nans:  0
type of data:  <class 'numpy.int32'>

 90. hasF1885                           :      2
Number of nans:  0
type of data:  <class 'nump

We have the following changes:
1. Remove the features with No. unique values == 600000 or 1, and drop not useful feature 
    600000: CompanyID,CompanyName,CompanyNumber
    1: CompanyStatus,Filled, LimitedPartershipsNumGenParterners, LimitedPartnershipsNumLimPartners, Status20190701
    not useful: imd, as we are using imdu instead of imd
    
    Note: This include CompanyName, which might be useful 
2. Convert Date into its duration:
    - dAccountsTimeGap = dAccountsNextDueDate - dAccountsLastMadeUpDate
    - dConfStmtTimeGap = dConfStmtNextDueDate - dConfStmtLastMadeUpDate 
    - dReturnsTimeGap = dReturnsNextDueDate - dReturnsLastMadeUpDate 
    #Not Sure about dPreviousName_1CONDATE and dIncorporationDate
3. For postcode-related field, delete pcd (postcode), and add:
    - OtherCompInPcd: how many other company in the postcode
    
#Keep the accounting data and the "hasF" data as binary indicator of estimation.
4. Detect the exponentially distributed feature: and take log (base 10)
    - All Accounting data with Field__ 
    - FillingID
5. Detect Outlier: 
    Upper = Q3 + 1.5*IQR
    Lower = Q1 - 1.5*IQR
    
Useful Feature lists:
    AccountsAccountCategory & CompanyNumber?
    All Mortgages fields:
        e.g. MortgagesNumMortCharges
    RegAddressCountry & RegAddressCounty
    SIC1,SIC2,SIC3
    dIncorporationDate
    eAccounts and eCompanyCategory
    hasGNotice 
    ru11ind

In [4]:
df_co["oseast1m"].unique()

array([411310., 392203., 529221., ..., 263278., 399432., 322950.])

In [5]:
def findRange(df_co):
    print("The range of the data")
    for n,c in enumerate(df_co.select_dtypes(include=['int32','int64','float64']).columns):
        print("{:3d}. {:<35}: {:>6}".format(n,c, (df_co[c].max() - df_co[c].dropna().min())))
        print()

In [6]:
findRange(df_co)

The range of the data
  0. CompanyId                          : 5925439

  1. CompanyNameCountNum                :     32

  2. CompanyNameCountX                  :      5

  3. CompanyNameLen                     :    109

  4. CompanyNameWordLen                 :     15

  5. Field1014                          : 329999890000.0

  6. Field1129                          : 263547732.0

  7. Field1522                          : 38121000.0

  8. Field1631                          : 423582131.0

  9. Field17                            : 2510357.0

 10. Field1865                          : 329999890000.0

 11. Field1871                          : 1000000000.0

 12. Field1885                          : 898167989.0

 13. Field1977                          : 18282768.0

 14. Field2298                          : 157092140.0

 15. Field2304                          : 15331834.0

 16. Field2316                          : 135881241.0

 17. Field2447                          : 55806997.0

 18. Field2

Try removing problematic features for now

In [7]:
# 1. Remove the features with No. unique values == 600000 or 1 
zero_info_features = ["CompanyId", "CompanyNumber","CompanyName"]
# Note here that the CompanyName is already included
only_one_value_features = ["CompanyStatus","Filled", "LimitedPartnershipsNumGenPartners",\
                           "LimitedPartnershipsNumLimPartners","Status20190701"]
not_useful_features = ["imd"]
df_curated = df_co.drop(zero_info_features + only_one_value_features + not_useful_features, axis=1)

In [8]:
# 2. Convert Date into its duration
drop_date_features = ["dAccountsNextDueDate","dConfStmtNextDueDate","dReturnsNextDueDate","dAccountsLastMadeUpDate",\
                      "dConfStmtLastMadeUpDate","dReturnsLastMadeUpDate"]
df_curated["dAccountsTimeGap"] = df_curated["dAccountsNextDueDate"] - df_curated["dAccountsLastMadeUpDate"]
df_curated["dConfStmtTimeGap"] = df_curated["dConfStmtNextDueDate"] - df_curated["dConfStmtLastMadeUpDate"] 
df_curated["dReturnsTimeGap"] = df_curated["dReturnsNextDueDate"] - df_curated["dReturnsLastMadeUpDate"] 
df_curated2 = df_curated.drop(drop_date_features,axis = 1)

In [9]:
# 3. Delete pcd (postcode), and add OtherCompInPcd: how many other company in the postcode
def NumOtherCompPcd(row): return pcdDict[row["pcd"]]

# Generate the pcd dictionary for fast checking
pcdDict = dict()
for i in df_curated2["pcd"]:
    pcdDict[i] = pcdDict.get(i, 0) + 1

df_curated2["OtherCompInPcd"] = df_curated2.apply (lambda row: NumOtherCompPcd(row),axis = 1)
df_curated3 = df_curated2.drop("pcd",axis = 1)

In [10]:
# 4. Filter out all the features with Field___ and Filing that might have exponential distribution
exponential_features = [x for x in df_curated3.columns if x[0] == 'F']
for feature in exponential_features:
    # This part works by approximating the log value
    # ! All the None/'space' needs to be changed to Nan before in order to remove try/except 
    try:
        df_curated3[feature]=np.log10(df_curated3[feature]+1)
    except:
        pass

In [11]:
# 5. Delete the outliers for each numerics value
for n,c in enumerate(df_curated3.select_dtypes(include=['int32','int64','float64']).columns):
    q_low = df_curated3[c].quantile(0.01)
    q_hi  = df_curated3[c].quantile(0.99)
    df_filtered = df_curated3[(df_curated3[c] < q_hi) & (df_curated3[c] > q_low)]

In [12]:
findRange(df_filtered)

The range of the data
  0. CompanyNameCountNum                :     32

  1. CompanyNameCountX                  :      5

  2. CompanyNameLen                     :     98

  3. CompanyNameWordLen                 :     15

  4. Field1014                          : 11.518513795114352

  5. Field1129                          : 8.420859284827868

  6. Field1522                          : 7.581164296034197

  7. Field1631                          : 8.626937631941892

  8. Field17                            : 6.399735660261015

  9. Field1865                          : 11.518513795114352

 10. Field1871                          : 9.000000000434294

 11. Field1885                          : 8.953357573089102

 12. Field1977                          : 7.262041972051498

 13. Field2298                          : 8.196154458714654

 14. Field2304                          : 7.185594136764475

 15. Field2316                          : 8.133159507812955

 16. Field2447                          : 7.

Remark: This process needs to be done before the preprocessing pipeline is finished